<a href="https://colab.research.google.com/github/ArMi1407/Consumo-de-gas-Argentina/blob/main/ETL_proceso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Extracción de datos
Los 24 Csv's originales se encuentran en el repositorio.
En esta notebook se realizó el tratamiento correspondiente para dejar los datos listos para ser utilizados.

In [1]:
#Importar librerías que serán utilizadas
import pandas as pd
import numpy as np
import os
import unicodedata
from google.colab import files
import io

In [2]:
"""Defino función para subir varios archivos CSV y convertirlos automáticamente en DataFrames, generando un diccionario donde
las claves son los nombres de archivo y los valores son los DataFrames."""
def upload_and_load_csvs():
    uploaded = files.upload()  # Puedo subir uno o varios CSV
    dataframes = {}

    for filename, content in uploaded.items():
        df_name = os.path.splitext(filename)[0]
        df = pd.read_csv(io.BytesIO(content))
        dataframes[df_name] = df
        print(f"✅ {df_name} cargado correctamente. Filas: {len(df)}")

    return dataframes

#Defino función para inspeccionar todos los DataFrames del diccionario generado
def inspect_all_dataframes(dataframe):
        print("="*80)
        print(f"📊 INSPECCIÓN DEL DATAFRAME: {name.upper()}")
        print("="*80)
        print("Primeras 5 filas del DataFrame:")
        print(df.head())
        print("\nInformación del DataFrame:")
        print(df.info())
        print("\nEstadísticas descriptivas del DataFrame:")
        print(df.describe())
        print("\nValores nulos en cada columna:")
        print(df.isnull().sum())
        print("\nNombres de las columnas:")
        print(df.columns.tolist())
        print("\nNúmero de filas duplicadas:")
        print(df.duplicated().sum())
        print("\nNúmero de valores únicos en cada columna:")
        print(df.nunique())
        print("\n\n")

In [3]:
#Aplicar la función de carga y conversión de csvs
uploaded_dataframes = upload_and_load_csvs()

Saving actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual.csv to actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual.csv
Saving bsas_temperatura.csv to bsas_temperatura.csv
Saving catamarca_temperatura.csv to catamarca_temperatura.csv
Saving chaco_temperatura.csv to chaco_temperatura.csv
Saving chubut_temperatura.csv to chubut_temperatura.csv
Saving cordoba_temperatura.csv to cordoba_temperatura.csv
Saving corrientes_temperatura.csv to corrientes_temperatura.csv
Saving entre_rios_temperatura.csv to entre_rios_temperatura.csv
Saving formosa_temperatura.csv to formosa_temperatura.csv
Saving jujuy_temperatura.csv to jujuy_temperatura.csv
Saving la_pampa_temperatura.csv to la_pampa_temperatura.csv
Saving la_rioja_temperatura.csv to la_rioja_temperatura.csv
Saving mendoza_temperatura.csv to mendoza_temperatura.csv
Saving misiones_temperatura.csv to misiones_temperatura.csv
Saving neuquen_temperatura.csv to neuquen_temperatura.csv
Saving r_negro_tempe

### Inspección de los datos

In [4]:
#Con un bucle for y la función definida se inspecciona cada dataframe
for name, df in uploaded_dataframes.items():
    inspect_all_dataframes(df)

📊 INSPECCIÓN DEL DATAFRAME: ACTIVIDAD-SALDOCOMERCIAL-RANGOS-EXPORTACION-EMPRESA-EXPORTADORA-MENSUAL
Primeras 5 filas del DataFrame:
  indice_tiempo  produccion_gas_natural     total  residencial  comercial  \
0    1996-01-01                 2504.98  1913.116      211.996     42.534   
1    1996-02-01                 2325.37  1731.663      216.713     43.851   
2    1996-03-01                 2735.07  2018.787      225.193     56.130   
3    1996-04-01                 2584.15  1935.116      351.159     73.160   
4    1996-05-01                 3006.48  2304.546      572.829     75.569   

   entes_oficiales  industria  centrales_electricas     sdb     gnc  metrogas  \
0            8.783    605.413               803.185  12.203  85.856   496.604   
1            9.499    626.625               656.703  14.482  75.903   398.279   
2           16.476    669.272               832.398  15.871  80.857   503.750   
3           17.779    658.115               612.963  18.109  88.107   458.635   


#2. Transformación


## CSV temperatura por provincias
Para la temperatura mensual por provincia.
A 23 de los 24 csv que cargados se les realizará la tranformación de los datos de manera que queden listos para el análisis y el modelado.
Las modificaciones serán las siguientes:
- Fecha: Los registros diarios serán agrupados para generar registros mensuales, además la columna 'date' será convertida a tipo de dato fecha y renombrada como 'indice_tiempo'.
- Luego de la agrupación mensual se calculará la temperatura promedio para cada mes
- Se eliminarán columnas que no serán utilizadas
- Se tratarán duplicados y nulos

###Transformación para la fecha

In [5]:
#Para el csv stgo_del_estero_temperatura transformo la columna 'date' en formato fecha y renombro columna
if 'stgo_del_estero_temperatura' in uploaded_dataframes:
    df_stgo_del_estero = uploaded_dataframes['stgo_del_estero_temperatura']
    df_stgo_del_estero['date'] = pd.to_datetime(df_stgo_del_estero['date'], errors='coerce', dayfirst=True)
    #Renombrar la columna 'date' como indice_tiempo
    df_stgo_del_estero = df_stgo_del_estero.rename(columns={'date': 'indice_tiempo'})

  #guardo en el diccionario
    uploaded_dataframes['stgo_del_estero_temperatura'] = df_stgo_del_estero

  #Mostrar el cambio
    print(df_stgo_del_estero.head(),
          df_stgo_del_estero.dtypes)

  indice_tiempo   tavg  tmin  tmax   prcp    pres     tsun
0    2019-01-01  27.00  21.1  32.9  390.0  1008.4  11040.0
1    2019-02-01  26.00  20.1  31.9   11.0  1010.7  14160.0
2    2019-03-01  23.15  18.0  28.3   77.0  1013.5   8400.0
3    2019-04-01  20.50  15.3  25.7   83.0  1014.0   8520.0
4    2019-05-01  16.65  11.2  22.1    9.0  1015.3   8820.0 indice_tiempo    datetime64[ns]
tavg                    float64
tmin                    float64
tmax                    float64
prcp                    float64
pres                    float64
tsun                    float64
dtype: object


In [6]:
#Para el resto de los csv (actividades) transformo la columna 'date' en formato fecha, agrupo los registros diarios a mensuales para cada df y renombro columna
"""Para no perder los datos, defino diccionario de funciones de agregación para cada columna:
- tavg, tmin, tmax, pres se promedian (media)
- prcp, tsun (precipitación y horas de sol) se suman"""
dict_agg = {
    'tavg': 'mean',
    'tmin': 'min',
    'tmax': 'max',
    'prcp': 'sum',
    'pres': 'mean',
    'tsun': 'sum',
    'snow': 'sum',
    'wdir': 'mean',
    'wspd': 'mean',
    'wpgt': 'max'
}

for name, df in uploaded_dataframes.items():
    if name not in ['actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual', 'stgo_del_estero_temperatura']:
        if 'date' in df.columns:
            # 1. Preparar la columna 'indice_tiempo'
            df['indice_tiempo'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True).dt.to_period('M').dt.to_timestamp()
            # 2.Agrupar por el nuevo índice de tiempo y agregar los datos, usando .agg() con y el diccionario
            df_agregado = df.groupby('indice_tiempo').agg(dict_agg).reset_index()

            # 3. Formatear la columna de tiempo a la cadena 'YYYY-MM-01'
            df_agregado['indice_tiempo'] = df_agregado['indice_tiempo']

            # 4. Reemplazar el DataFrame original en el diccionario
            uploaded_dataframes[name] = df_agregado

            # Mostrar los cambios
            print(f"DataFrame agregado: {name}")
            print(df_agregado.head())

#Guardar en el diccionario
for name, df in uploaded_dataframes.items():
  if name != 'actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual'and name!= 'stgo_del_estero_temperatura':
    uploaded_dataframes[name] = df
    #Mostrar los cambios
    print(f"DataFrame: {name}")
    print(df.head())


DataFrame agregado: bsas_temperatura
  indice_tiempo       tavg  tmin  tmax  prcp         pres  tsun  snow  wdir  \
0    2019-01-01  18.450000   3.0  32.2   0.0  1014.433333   0.0   5.0   NaN   
1    2019-02-01  19.000000   6.5  32.7   2.0  1014.600000   0.0   0.0   NaN   
2    2019-03-01  16.791667   2.4  32.7   8.9  1017.550000   0.0   0.0   NaN   
3    2019-04-01  17.200000   3.5  29.2   0.0  1016.941667   0.0   0.0   NaN   
4    2019-05-01  17.333333   2.0  29.7   0.0  1018.783333   0.0   1.0   NaN   

        wspd  wpgt  
0   9.558333   NaN  
1  10.341667   NaN  
2   9.483333   NaN  
3   8.766667   NaN  
4   9.625000   NaN  
DataFrame agregado: catamarca_temperatura
  indice_tiempo       tavg  tmin  tmax  prcp  pres  tsun  snow  wdir  wspd  \
0    2019-01-01  22.900000  10.9  31.4   1.0   NaN   0.0   0.0   NaN   NaN   
1    2019-02-01  22.427273   7.5  40.0   8.2   NaN   0.0   0.0   NaN   NaN   
2    2019-03-01  21.190909   3.8  39.5  21.1   NaN   0.0   0.0   NaN   NaN   
3    201

### Eliminar columnas

In [7]:
#Eliminar columnas que no se usarán
columns_to_drop = ['tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']

for name, df in uploaded_dataframes.items():
    if name != 'actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual':
        cols_existentes = [col for col in columns_to_drop if col in df.columns]

        if cols_existentes:  # si hay al menos una columna para eliminar
            df.drop(columns=cols_existentes, inplace=True)
            print(f"✅ DataFrame '{name}': se eliminaron las columnas {cols_existentes}")
            print(df.info(), "\n")
        else:
            print(f"⚠️ DataFrame '{name}': no contiene ninguna de las columnas a eliminar.\n")


#Mostrar cambio
for name, df in uploaded_dataframes.items():
  if name != 'actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual':
    print(f"DataFrame: {name}")
    print(df.info())
    print('\n')

✅ DataFrame 'bsas_temperatura': se eliminaron las columnas ['tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   indice_tiempo  60 non-null     datetime64[ns]
 1   tavg           60 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.1 KB
None 

✅ DataFrame 'catamarca_temperatura': se eliminaron las columnas ['tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   indice_tiempo  60 non-null     datetime64[ns]
 1   tavg           60 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1

###Tratamiento de datos

In [8]:
#Verificar si hay Nan o nulos
for name, df in uploaded_dataframes.items():
  if name != 'actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual':
    print(f"DataFrame: {name}")
    print(df.isnull().sum())
    print('\n')

DataFrame: bsas_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: catamarca_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: chaco_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: chubut_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: cordoba_temperatura
indice_tiempo     0
tavg             12
dtype: int64


DataFrame: corrientes_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: entre_rios_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: formosa_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: jujuy_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: la_pampa_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: la_rioja_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: mendoza_temperatura
indice_tiempo    0
tavg             9
dty

In [9]:
#Mirar las estadísticas
for name, df in uploaded_dataframes.items():
  if name != 'actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual':
    print(f"DataFrame: {name}")
    print(df.describe())

DataFrame: bsas_temperatura
             indice_tiempo       tavg
count                   60  60.000000
mean   2021-06-16 06:24:00  18.663750
min    2019-01-01 00:00:00  16.716667
25%    2020-03-24 06:00:00  18.097917
50%    2021-06-16 00:00:00  18.654167
75%    2022-09-08 12:00:00  19.000000
max    2023-12-01 00:00:00  20.800000
std                    NaN   0.895006
DataFrame: catamarca_temperatura
             indice_tiempo       tavg
count                   60  60.000000
mean   2021-06-16 06:24:00  22.692040
min    2019-01-01 00:00:00  19.037500
25%    2020-03-24 06:00:00  21.595000
50%    2021-06-16 00:00:00  22.394886
75%    2022-09-08 12:00:00  23.778472
max    2023-12-01 00:00:00  26.300000
std                    NaN   1.568200
DataFrame: chaco_temperatura
             indice_tiempo       tavg
count                   60  60.000000
mean   2021-06-16 06:24:00  21.588678
min    2019-01-01 00:00:00  15.533333
25%    2020-03-24 06:00:00  20.555000
50%    2021-06-16 00:00:00  21.60833

No se encontraron valores que necesitaran ser reemplazados*, pero de haberse encontrado:
 - En 'indice_tiempo' (columna en formato fecha) se hubiera utilizado el método de imputación **.fillna(method='ffill')** para que los NaT(Not a Time) en una fila tomen el valor de la fila anterior.
 - En la columna 'tavg' para los csv cordoba (12) y mendoza(9), se imputará con la mediana.

*Para los valores que necesitan ser reemplazados, Pandas cuenta todos estos indicadores de "falta de valor" (NaN, None, NaT), utilizándose el mismo método para encontrarlos **df.isnull().sum()**

In [10]:
#Imputar datos faltantes para el csv cordoba_temperatura y mendoza_temperatura
for name, df in uploaded_dataframes.items():
  if name in ['cordoba_temperatura', 'mendoza_temperatura']:
    df['tavg'].fillna(df['tavg'].median(), inplace=True)
    print(f"DataFrame: {name}")
    print(df.isnull().sum())
    print('\n')

DataFrame: cordoba_temperatura
indice_tiempo    0
tavg             0
dtype: int64


DataFrame: mendoza_temperatura
indice_tiempo    0
tavg             0
dtype: int64




/tmp/ipython-input-3197031113.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tavg'].fillna(df['tavg'].median(), inplace=True)


###Merge de los csv

In [11]:
"""Union de los csv en un solo que contendrá las temperaturas de todas las provincias
la clave de unión será la columna indice_tiempo"""

"""NOTA: La función reduce de Python, que se importa desde el módulo functools,
sirve para aplicar repetidamente una función a los elementos de una secuencia
(como una lista) para reducir la secuencia a un único valor acumulado."""
from functools import reduce

# 1. Renombrar las columnas en cada DataFrame (excepto la clave de unión)
clave_union = 'indice_tiempo'
dfs_renombrados = {}

for name, df in uploaded_dataframes.items():
    # Creamos un nuevo diccionario de mapeo {nombre_viejo: nombre_nuevo}
    columnas_a_renombrar = {}

    for col in df.columns:
        if col != clave_union:
            # Creamos un nuevo nombre: 'tavg' -> 'tavg_DF_nombre'
            columnas_a_renombrar[col] = f"{col}_{name}"

    # Aplicamos el renombramiento y guardamos el DF limpio
    dfs_renombrados[name] = df.rename(columns=columnas_a_renombrar)


# 2. Excluir el DataFrame de tratamiento aparte y crear la lista de DataFrames a fusionar
nombre_df_excluido = 'actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual'

lista_de_dfs_a_fusionar = [
    df for name, df in dfs_renombrados.items()
    if name != nombre_df_excluido
]


# 3. Aplicar la fusión (Merge) con las columnas renombradas
tipo_union = 'outer'

df_final = reduce(
    lambda left, right: pd.merge(left, right, on=clave_union, how=tipo_union),
    lista_de_dfs_a_fusionar
)

print(f"¡Fusión exitosa con {len(lista_de_dfs_a_fusionar)} DataFrames y columnas únicas!")
print(df_final.head())

¡Fusión exitosa con 23 DataFrames y columnas únicas!
  indice_tiempo  tavg_bsas_temperatura  tavg_catamarca_temperatura  \
0    2019-01-01              18.450000                   22.900000   
1    2019-02-01              19.000000                   22.427273   
2    2019-03-01              16.791667                   21.190909   
3    2019-04-01              17.200000                   21.280000   
4    2019-05-01              17.333333                   19.037500   

   tavg_chaco_temperatura  tavg_chubut_temperatura  tavg_cordoba_temperatura  \
0               18.925000                13.241667                 19.216667   
1               20.625000                12.650000                 19.216667   
2               23.400000                11.100000                 19.216667   
3               18.171429                11.650000                 19.216667   
4               18.762500                13.054545                 19.216667   

   tavg_corrientes_temperatura  tavg_entre_ri

In [12]:
#Visualizar el resultado del merge
df_final.tail()

,indice_tiempo,tavg_bsas_temperatura,tavg_catamarca_temperatura,tavg_chaco_temperatura,tavg_chubut_temperatura,tavg_cordoba_temperatura,tavg_corrientes_temperatura,tavg_entre_rios_temperatura,tavg_formosa_temperatura,tavg_jujuy_temperatura,...,tavg_neuquen_temperatura,tavg_r_negro_temperatura,tavg_salta_temperatura,tavg_sanjuan_temperatura,tavg_sanluis_temperatura,tavg_santafe_temperatura,tavg_sta_cruz_temperatura,tavg_stgo_del_estero_temperatura,tavg_t_d_fuego_temperatura,tavg_tucuman_temperatura
55,2023-08-01,20.450000,24.163636,21.60,15.525000,21.300000,22.741667,20.608333,24.163636,21.075000,...,17.541667,16.491667,19.520,20.633333,20.683333,20.525000,8.208333,17.35,7.325000,19.120000
56,2023-09-01,20.800000,23.788889,24.56,15.825000,21.466667,22.441667,20.925000,23.788889,21.183333,...,17.083333,16.775000,17.975,20.916667,20.933333,20.975000,7.608333,20.55,6.400000,24.550000
57,2023-10-01,20.625000,25.340000,24.25,14.541667,22.516667,23.975000,21.400000,25.340000,21.600000,...,16.691667,16.066667,18.940,21.583333,20.883333,22.183333,7.591667,23.65,6.366667,19.066667
58,2023-11-01,20.058333,26.300000,19.50,13.575000,21.166667,24.950000,21.000000,26.300000,22.275000,...,15.408333,15.700000,18.000,20.533333,19.241667,20.925000,7.200000,26.20,5.691667,20.228571
59,2023-12-01,18.666667,24.322222,21.80,12.516667,19.333333,21.900000,18.650000,24.322222,21.566667,...,14.158333,14.183333,17.180,18.458333,17.958333,18.916667,7.591667,27.10,6.483333,22.366667


### Wide a Long

In [13]:
"""Por cuestiones de visualización práctica, se cambiará de manera que pase de
wide---> a long y las provincias se les quitara el y prefijo 'tavg' y subfijo 'temperatura'"""
# Melt: pasar de formato wide a long, la nueva columna se llamará temp_promedio
df_merged_long = pd.melt(df_final, id_vars=['indice_tiempo'], var_name='provincia', value_name='temp_promedio')
"""Quitar el subfijo '_temperatura' y el prefijo 'tavg' de los nombres de
provincias para que solo queden los nombres de las provincias en la nueva columna"""
df_merged_long['provincia'] = df_merged_long['provincia'].str.replace('_temperatura', '').str.replace('tavg_', '')
# Mostrar resultado
df_merged_long.head()


,indice_tiempo,provincia,temp_promedio
0,2019-01-01,bsas,18.450000
1,2019-02-01,bsas,19.000000
2,2019-03-01,bsas,16.791667
3,2019-04-01,bsas,17.200000
4,2019-05-01,bsas,17.333333


In [14]:
df_merged_long.tail()

,indice_tiempo,provincia,temp_promedio
1375,2023-08-01,tucuman,19.120000
1376,2023-09-01,tucuman,24.550000
1377,2023-10-01,tucuman,19.066667
1378,2023-11-01,tucuman,20.228571
1379,2023-12-01,tucuman,22.366667


###Nuevas columnas: Estación y GDC (Grados/Dia de Calefacción)

Se crearán nuevas columnas que aportarán gran valor y contexto a los datos.
- La primera será Estación, la cual será asignada mediante una función que utilizará la columna indice_tiempo
- La segunda será GDC, de la misma manera mediante una función, que tomará la temperatura promedio.
  Los Grados Día de Calefacción (GDC): Se usan para calcular el consumo de gas para calefacción en los meses fríos, siendo un tipo de medida que cuantifica la demanda de energía para calentar un edificio, relacionando las temperaturas exteriores con una temperatura de confort deseada.

In [15]:
#Definir las funciones
#Defino función para asignar la estación
def asignar_estacion(fecha):
    mes = fecha.month
    if 12==mes or 1==mes or 2==mes:
        return 'Verano'
    elif 3==mes or 4==mes or 5==mes:
        return 'Otoño'
    elif 6==mes or 7==mes or 8==mes:
        return 'Invierno'
    elif 9==mes or 10==mes or 11==mes:
        return 'Primavera'

"""Defino función que calcula el gdc.
IMPORTANTE, un valor mayor a 18 es igual a 0
Se calculan como la suma de las diferencias entre una temperatura base de calefacción y
la temperatura media diaria exterior,
considerando únicamente los días en que la temperatura media es inferior a la temperatura base.
La temperatura base es 18
La temperatura media diaria se toma 30 como valor"""
def calcular_gdc_mensual_aprox(temp_media_mensual):
    """
    Calcula una APROXIMACIÓN del GDC usando la temperatura media mensual.
    La fórmula es: max(0, T_base - T_media) * 30 días.
    """
    temp_base = 18
    dias_del_mes_aprox = 30  #30 como valor constante para la aproximación

    # 1. Calcular la diferencia (T_base - T_media)
    diferencia_temperatura = temp_base - temp_media_mensual

    # 2. Aplicar la regla: si la diferencia es negativa, es 0
    gdc_unitario = max(0, diferencia_temperatura)

    # 3. Multiplicar por los días para obtener el valor acumulado mensual
    gdc_total_mes = gdc_unitario * dias_del_mes_aprox

    return gdc_total_mes

In [16]:
#Generar columna estación, con los datos correspondientes
df_merged_long['estacion'] = df_merged_long['indice_tiempo'].apply(asignar_estacion)

#Mostrar resultados
df_merged_long.head()

,indice_tiempo,provincia,temp_promedio,estacion
0,2019-01-01,bsas,18.450000,Verano
1,2019-02-01,bsas,19.000000,Verano
2,2019-03-01,bsas,16.791667,Otoño
3,2019-04-01,bsas,17.200000,Otoño
4,2019-05-01,bsas,17.333333,Otoño


In [17]:
"""Generar la columna gdc y los valores correspondientes (Recordando que un gdc
mayor a 18 es igual a 0)"""
df_merged_long['gdc'] = df_merged_long['temp_promedio'].apply(calcular_gdc_mensual_aprox)

#Mostrar resultados
df_merged_long.head()

,indice_tiempo,provincia,temp_promedio,estacion,gdc
0,2019-01-01,bsas,18.450000,Verano,0.00
1,2019-02-01,bsas,19.000000,Verano,0.00
2,2019-03-01,bsas,16.791667,Otoño,36.25
3,2019-04-01,bsas,17.200000,Otoño,24.00
4,2019-05-01,bsas,17.333333,Otoño,20.00


## Transformación actividades del gas

El csv cargado (Denominado 'actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual') brinda información a nivel mensual sobre el consumo de gas natural a nivel nacional, residencial, por distribuidora y entre otros.
En el tratamiento a realizar se incluye:
- La transformación de la columna indice_tiempo a date y el establecimiento del periodo de 2019-2023
- Eliminación de columnas que no serán utilizadas.
- Agrupación de columnas para generar una nueva.

### Transformación para la fecha

In [18]:
#Conversión de dato para el csv 'actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual'
if 'actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual' in uploaded_dataframes:
  df_actividad = uploaded_dataframes['actividad-saldocomercial-rangos-exportacion-empresa-exportadora-mensual']
  df_actividad['indice_tiempo'] = pd.to_datetime(df_actividad['indice_tiempo'], errors='coerce')

#Mostrar cambio
print(df_actividad.head(),
      df_actividad.dtypes)

  indice_tiempo  produccion_gas_natural     total  residencial  comercial  \
0    1996-01-01                 2504.98  1913.116      211.996     42.534   
1    1996-02-01                 2325.37  1731.663      216.713     43.851   
2    1996-03-01                 2735.07  2018.787      225.193     56.130   
3    1996-04-01                 2584.15  1935.116      351.159     73.160   
4    1996-05-01                 3006.48  2304.546      572.829     75.569   

   entes_oficiales  industria  centrales_electricas     sdb     gnc  metrogas  \
0            8.783    605.413               803.185  12.203  85.856   496.604   
1            9.499    626.625               656.703  14.482  75.903   398.279   
2           16.476    669.272               832.398  15.871  80.857   503.750   
3           17.779    658.115               612.963  18.109  88.107   458.635   
4           23.653    645.976               748.281  22.857  91.773   562.713   

   gas_natural_fenosa  distrib_gas_del_centro_ecog

In [19]:
#Establecer periodo de tiempo
df_actividad = df_actividad[(df_actividad['indice_tiempo'] >= '2019-01-01') & (df_actividad['indice_tiempo'] <= '2023-12-01')]

#Mostrar cambio
print(df_actividad.head(),
      df_actividad.tail())

    indice_tiempo  produccion_gas_natural     total  residencial  comercial  \
276    2019-01-01             4000.194452  3267.863      306.257     82.965   
277    2019-02-01             3770.580137  3147.144      277.859     80.342   
278    2019-03-01             4014.104313  3284.154      405.178     99.218   
279    2019-04-01             3948.832523  3279.103      498.612     95.504   
280    2019-05-01             4245.779178  3879.933      958.250    130.692   

     entes_oficiales  industria  centrales_electricas     sdb      gnc  \
276           11.177    971.393              1480.435  63.461  193.813   
277           10.635    970.018              1425.965  59.560  183.953   
278           15.609   1007.868              1231.684  52.438  206.137   
279           21.842    990.284              1098.785  66.781  198.856   
280           45.922    961.941              1222.893  92.620  208.497   

     metrogas  gas_natural_fenosa  distrib_gas_del_centro_ecogas  \
276   634.21

### Columnas

In [20]:
#Se descartarán las siguientes columnas definidas en la lista columna_excluir
columna_excluir=['metrogas', 'gas_natural_fenosa', 'distrib_gas_del_centro_ecogas', 'distrib_gas_cuyana_ecogas', 'litoral_gas', 'gasnea', 'redengas', 'gasnor', 'camuzzi_gas_pampeana', 'camuzzi_gas_del_sur']
df_actividad.drop(columns=columna_excluir, inplace=True)

/tmp/ipython-input-690759142.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actividad.drop(columns=columna_excluir, inplace=True)


In [21]:
#Agrupamiento de columnas

"""Lista de columnas que serán agrupadas en una nueva columna llamada 'Otros sectores',
puesto que no serán analizadas individualmente"""
columnas_sumar = ['comercial', 'entes_oficiales', 'industria', 'centrales_electricas', 'sdb', 'gnc']

# Crear la nueva columna con la suma fila por fila
df_actividad['Otros sectores'] = df_actividad[columnas_sumar].sum(axis=1)

#Eliminar las columnas que fueron sumadas en la nueva columna 'Otros sectores'
df_actividad.drop(columns=columnas_sumar, inplace=True)

#Mostrar cambios en el df
print(df_actividad.head(),
      df_actividad.tail())

    indice_tiempo  produccion_gas_natural     total  residencial  \
276    2019-01-01             4000.194452  3267.863      306.257   
277    2019-02-01             3770.580137  3147.144      277.859   
278    2019-03-01             4014.104313  3284.154      405.178   
279    2019-04-01             3948.832523  3279.103      498.612   
280    2019-05-01             4245.779178  3879.933      958.250   

     Otros sectores  
276        2803.244  
277        2730.473  
278        2612.954  
279        2472.052  
280        2662.565       indice_tiempo  produccion_gas_natural     total  residencial  \
331    2023-08-01             4477.024219  4469.913     1445.818   
332    2023-09-01             4296.923314  4014.886     1191.804   
333    2023-10-01             3905.392633  3456.851      735.477   
334    2023-11-01             3789.920718  3364.695      551.415   
335    2023-12-01             3539.010723  3163.924      383.230   

     Otros sectores  
331        2791.544  
332   

/tmp/ipython-input-2071361210.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actividad['Otros sectores'] = df_actividad[columnas_sumar].sum(axis=1)
/tmp/ipython-input-2071361210.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actividad.drop(columns=columnas_sumar, inplace=True)


In [22]:
#Verificar presencia de nulos y duplicados
print(df_actividad.isnull().sum(),
      df_actividad.duplicated().sum())

indice_tiempo             0
produccion_gas_natural    0
total                     0
residencial               0
Otros sectores            0
dtype: int64 0


#3. Merge del csv final
Se unirá el csv con los datos de las provincias (temperatura, estación y gdc) con el de actividades del gas para dar lugar al csv final que será utilizado para análisis y modelado

In [23]:
#La clave de unión será la columna 'indice_tiempo'
df_final_merge = pd.merge(df_merged_long, df_actividad, on='indice_tiempo', how='left')

#Mostrar el nuevo df
df_final_merge.head()

,indice_tiempo,provincia,temp_promedio,estacion,gdc,produccion_gas_natural,total,residencial,Otros sectores
0,2019-01-01,bsas,18.450000,Verano,0.00,4000.194452,3267.863,306.257,2803.244
1,2019-02-01,bsas,19.000000,Verano,0.00,3770.580137,3147.144,277.859,2730.473
2,2019-03-01,bsas,16.791667,Otoño,36.25,4014.104313,3284.154,405.178,2612.954
3,2019-04-01,bsas,17.200000,Otoño,24.00,3948.832523,3279.103,498.612,2472.052
4,2019-05-01,bsas,17.333333,Otoño,20.00,4245.779178,3879.933,958.250,2662.565


In [24]:
#Verificar nulos y duplicados
print(df_final_merge.isnull().sum(),
      df_final_merge.duplicated().sum())

indice_tiempo             0
provincia                 0
temp_promedio             0
estacion                  0
gdc                       0
produccion_gas_natural    0
total                     0
residencial               0
Otros sectores            0
dtype: int64 0


In [25]:
#Guardar csv
df_final_merge.to_csv('df_final_merge.csv', index=False)

files.download('df_final_merge.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>